In [11]:
import requests
import pandas as pd

dfs = []
# , 2018 , 2019, 2020, 2021, 2022, 2023
for anio_inicio in [2021, 2022, 2023]:
    anio_fin = anio_inicio + 1

    url = (
        "https://www.datos.gov.co/resource/s54a-sgyg.json?"
        f"$where=fechaobservacion >= '{anio_inicio}-01-01' "
        f"AND fechaobservacion < '{anio_fin}-01-01' "
        "AND departamento in ('CUNDINAMARCA','NARIÑO')"
        "&$limit=3600000" 
    )

    print("Descargando año:", anio_inicio)
    response = requests.get(url)
    response.raise_for_status()

    data = response.json()
    df_tmp = pd.DataFrame(data)
    print("Filas descargadas:", df_tmp.shape[0])

    dfs.append(df_tmp)

# Unimos todos los años
df_precipitacion1 = pd.concat(dfs, ignore_index=True)

print("Total filas:", df_precipitacion1.shape)
print(df_precipitacion1.head())


Descargando año: 2021
Filas descargadas: 2127316
Descargando año: 2022
Filas descargadas: 2048081
Descargando año: 2023
Filas descargadas: 1788193
Total filas: (5963590, 12)
  codigoestacion codigosensor         fechaobservacion valorobservado  \
0     0021209920         0240  2021-01-01T00:00:00.000              0   
1     0035025501         0240  2021-01-01T00:00:00.000              0   
2     0024015506         0240  2021-01-01T00:00:00.000              0   
3     0023065505         0240  2021-01-01T00:00:00.000              0   
4     0021207702         0240  2021-01-01T00:00:00.000              0   

          nombreestacion  departamento  municipio zonahidrografica  \
0     SANTA ROSITA - AUT  CUNDINAMARCA     SUESCA   ALTO MAGDALENA   
1         QUETAME  - AUT  CUNDINAMARCA    QUETAME             META   
2       CAPELLANIA - CAR  CUNDINAMARCA    FÚQUENE         SOGAMOSO   
3        CAPARRAPI - CAR  CUNDINAMARCA  CAPARRAPÍ  MEDIO MAGDALENA   
4  GRANJA BOSCONIA - CAR  CUNDINAMARC

In [12]:
# Guardar CSV
df_precipitacion1.to_csv("precipitaciones2021_2023.csv", index=False, encoding="utf-8")

In [29]:
# Consolidar precipitaciones por rangos de años
df1 = pd.read_csv("precipitaciones2015_2017.csv")
df2 = pd.read_csv("precipitaciones2018_2020.csv")
df3 = pd.read_csv("precipitaciones2021_2023.csv")

df_precipitaciones = pd.concat([df1, df2, df3], ignore_index=True)
df_precipitaciones= df_precipitaciones[df_precipitaciones["descripcionsensor"] != "GPRS - PRECIPITACIÓN"]

print(df_precipitaciones.shape)
print(df_precipitaciones.head())

df_precipitaciones.to_csv("precipitaciones2015_2023.csv", index=False, encoding="utf-8")

(19243211, 12)
   codigoestacion  codigosensor         fechaobservacion  valorobservado  \
0        21195190           240  2015-01-01T00:00:00.000             0.0   
1        23037010           240  2015-01-01T00:00:00.000             0.0   
2        24017330           240  2015-01-01T00:00:00.000             0.0   
3        21206990           240  2015-01-01T00:00:00.000             0.0   
4        21206930           240  2015-01-01T00:00:00.000             0.0   

           nombreestacion  departamento      municipio zonahidrografica  \
0             PASCA - AUT  CUNDINAMARCA          PASCA   ALTO MAGDALENA   
1     PUERTO SALGAR - AUT  CUNDINAMARCA  PUERTO SALGAR  MEDIO MAGDALENA   
2      PTE LA BALSA - CAR  CUNDINAMARCA    LENGUAZAQUE         SOGAMOSO   
3         TIBAITATA - AUT  CUNDINAMARCA       MOSQUERA   ALTO MAGDALENA   
4  PARAMO GUERRERO  - AUT  CUNDINAMARCA      ZIPAQUIRÁ   ALTO MAGDALENA   

    latitud   longitud descripcionsensor unidadmedida  
0  4.310111 -74.31175

In [30]:
print(df_precipitaciones.duplicated().sum())

388586


In [31]:
df_precipitaciones= df_precipitaciones.drop_duplicates()
print(df_precipitaciones.shape)

(18854625, 12)


In [ ]:
# Seleccionar columnas de interés
df_precipitacion = df_precipitaciones[[
    "departamento",
    "municipio",
    "fechaobservacion",
    "valorobservado"
]].copy()

# Convertir fecha a datetime
df_precipitacion["fechaobservacion"] = pd.to_datetime(
    df_precipitacion["fechaobservacion"], errors="coerce"
)

# Crear columna solo con la fecha
df_precipitacion["fecha_dia"] = df_precipitacion["fechaobservacion"].dt.date

# Agrupar por departamento, municipio y día
df_diaria = (
    df_precipitacion
    .groupby(["departamento", "municipio", "fecha_dia"], as_index=False)
    .agg(precipitacion_diaria=("valorobservado", "mean"))
)

# Crear columna año-mes (YYYY-MM)
df_diaria["fecha_dia"] = pd.to_datetime(df_diaria["fecha_dia"])
df_diaria["mes"] = df_diaria["fecha_dia"].dt.strftime('%Y-%m')

# Convertir precipitación a numérico
df_diaria["precipitacion_diaria"] = pd.to_numeric(
    df_diaria["precipitacion_diaria"], errors="coerce"
)

# Agrupar por departamento, municipio y mes para sacar precipitación acumulada mensual
df_precipitacion_acum = (
    df_diaria
    .groupby(["departamento", "municipio", "mes"], as_index=False)
    .agg(
        precipitacion_acumulada=("precipitacion_diaria", "sum")
    )
)

print(df_precipitacion_acum.head())
print(df_precipitacion_acum.shape)
print(df_precipitacion_acum.describe())


C:\Users\tatia\AppData\Local\Temp\ipykernel_4564\1604910169.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_precipitacion["fechaobservacion"] = pd.to_datetime(


   departamento municipio  anio  mes  precipitacion_acumulada
0  CUNDINAMARCA  ANAPOIMA  2018    6                      4.8
1  CUNDINAMARCA  ANAPOIMA  2018    7                      1.0
2  CUNDINAMARCA  ANAPOIMA  2018    9                     28.8
3  CUNDINAMARCA  ANAPOIMA  2018   10                    179.2
4  CUNDINAMARCA  ANAPOIMA  2019    3                    190.0
(4442, 5)
              anio          mes  precipitacion_acumulada
count  4442.000000  4442.000000              4442.000000
mean   2019.086673     6.454075               135.609526
std       2.249052     3.483584               668.629263
min    2015.000000     1.000000                 0.000000
25%    2017.000000     3.000000                21.100000
50%    2019.000000     6.000000                64.400000
75%    2021.000000     9.000000               134.400000
max    2023.000000    12.000000             30726.500000


In [34]:
# Guardar CSV
df_precipitacion_acum.to_csv("precipitaciones.csv", index=False, encoding="utf-8")